Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [2]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import load_workbook
from yahoo_query import *

In [11]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

# stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
# stock_list = list(set(tickers[3:] + stock_list))

cad_names = pd.read_csv('cad_names.csv')['Symbol'].tolist()
us_names = pd.read_csv('us_names.csv')['Symbol'].tolist()
start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info_quarter = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
    
    annual_info = yahoo_data.cashFlowStatementAnnual.join(yahoo_data.incomeStatementAnnual.drop(['netIncome','maxAge'],
                                                                                                  axis = 1),
                                                           rsuffix='_income').join(yahoo_data.balanceSheetAnnual,
                                                                                   rsuffix = '_balance')
#     earnings_info['earnBeatsBefore'] = 0
#     earnings_info['earnMissBefore'] = 0

#     for idx, row in earnings_info.iterrows():
#         earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
#         earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
#     earnings_info = earnings_info.shift(1)


#     earnings_moves = past_earnings(ticker).sort_index()
#     earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
#                                     (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


#     earnings_df = pd.concat([earnings_info.reset_index(), 
#                              earnings_moves.reset_index()], axis = 1)
    earnings_info_quarter.columns = ['quarter' if col == 'index' else col for col in earnings_info_quarter.columns.tolist()]
    earnings_info_quarter['Underlying'] = ticker
    annual_info.columns = ['year' if col == 'index' else col for col in annual_info.columns.tolist()]
    annual_info['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_info_quarter, annual_info, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    annual_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            annual_lst.append(curr_annual_info)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    annual_lst.append(curr_annual_info)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    annual_df = pd.concat(annual_lst, axis = 0)
    #earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, annual_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [23]:
# earnings_df, annual_df, keyStats_df, failed_list = download_yahoo_data(us_names, retries = 10)

# datenow = dt.datetime.today().strftime("%Y-%m-%d")
# earnings_df.to_csv('us_quarterly_{}.csv'.format(datenow))
# annual_df.to_csv('us_annual_{}.csv'.format(datenow))
# keyStats_df.to_csv('us_keystats_{}.csv'.format(datenow))


In [22]:
earnings_df, annual_df, keyStats_df, failed_list = download_yahoo_data(cad_names, retries = 10)

datenow = dt.datetime.today().strftime("%Y-%m-%d")
earnings_df.to_csv('cad_quarterly_{}.csv'.format(datenow))
annual_df.to_csv('cad_annual_{}.csv'.format(datenow))
keyStats_df.to_csv('cad_keystats_{}.csv'.format(datenow))

Failed: AAB.TO
0.03% Completed
1 total failures
Accepted: AAV.TO
0.05% Completed
1 total failures
Accepted: ABT.TO
0.08% Completed
1 total failures
Accepted: ABX.TO
0.10% Completed
1 total failures
Accepted: AC.TO
0.13% Completed
1 total failures
Accepted: ACB.TO
0.15% Completed
1 total failures
Failed: ACB.WT.TO
0.18% Completed
2 total failures
Failed: ACD.TO
0.20% Completed
3 total failures
Failed: ACO.X.TO
0.23% Completed
4 total failures
Failed: ACO.Y.TO
0.26% Completed
5 total failures
Accepted: ACQ.TO
0.28% Completed
5 total failures
Failed: ACR.UN.TO
0.31% Completed
6 total failures
Failed: ACZ.UN.TO
0.33% Completed
7 total failures
Accepted: AD.TO
0.36% Completed
7 total failures
Failed: ADC.UN.TO
0.38% Completed
8 total failures
Accepted: ADN.TO
0.41% Completed
8 total failures
Failed: ADW.A.TO
0.43% Completed
9 total failures
Failed: ADW.B.TO
0.46% Completed
10 total failures
Failed: AEF.TO
0.49% Completed
11 total failures
Failed: AEF.WT.TO
0.51% Completed
12 total failures


Failed: BCE.PR.E.TO
4.07% Completed
115 total failures
Failed: BCE.PR.F.TO
4.09% Completed
116 total failures
Failed: BCE.PR.G.TO
4.12% Completed
117 total failures
Failed: BCE.PR.H.TO
4.14% Completed
118 total failures
Failed: BCE.PR.I.TO
4.17% Completed
119 total failures
Failed: BCE.PR.J.TO
4.20% Completed
120 total failures
Failed: BCE.PR.K.TO
4.22% Completed
121 total failures
Failed: BCE.PR.L.TO
4.25% Completed
122 total failures
Failed: BCE.PR.M.TO
4.27% Completed
123 total failures
Failed: BCE.PR.N.TO
4.30% Completed
124 total failures
Failed: BCE.PR.O.TO
4.32% Completed
125 total failures
Failed: BCE.PR.Q.TO
4.35% Completed
126 total failures
Failed: BCE.PR.R.TO
4.37% Completed
127 total failures
Failed: BCE.PR.S.TO
4.40% Completed
128 total failures
Failed: BCE.PR.T.TO
4.43% Completed
129 total failures
Failed: BCE.PR.Y.TO
4.45% Completed
130 total failures
Failed: BCE.PR.Z.TO
4.48% Completed
131 total failures
Accepted: BCI.TO
4.50% Completed
131 total failures
Accepted: BDI

Failed: CBH.TO
8.01% Completed
247 total failures
Accepted: CBL.TO
8.03% Completed
247 total failures
Failed: CBN.TO
8.06% Completed
248 total failures
Failed: CBO.TO
8.08% Completed
249 total failures
Failed: CBT.UN.TO
8.11% Completed
250 total failures
Accepted: CCA.TO
8.14% Completed
250 total failures
Failed: CCI.UN.TO
8.16% Completed
251 total failures
Failed: CCL.A.TO
8.19% Completed
252 total failures
Failed: CCL.B.TO
8.21% Completed
253 total failures
Failed: CCM.TO
8.24% Completed
254 total failures
Accepted: CCO.TO
8.26% Completed
254 total failures
Failed: CCS.PR.C.TO
8.29% Completed
255 total failures
Failed: CCX.TO
8.31% Completed
256 total failures
Failed: CCZ.TO
8.34% Completed
257 total failures
Failed: CCZ.PR.A.TO
8.37% Completed
258 total failures
Failed: CDAY.TO
8.39% Completed
259 total failures
Failed: CDD.UN.TO
8.42% Completed
260 total failures
Failed: CDH.TO
8.44% Completed
261 total failures
Accepted: CDV.TO
8.47% Completed
261 total failures
Failed: CDZ.TO
8.4

Accepted: CVE.TO
12.00% Completed
348 total failures
Failed: CVG.TO
12.02% Completed
349 total failures
Accepted: CWB.TO
12.05% Completed
349 total failures
Failed: CWB.PR.B.TO
12.07% Completed
350 total failures
Failed: CWB.PR.C.TO
12.10% Completed
351 total failures
Failed: CWI.TO
12.13% Completed
352 total failures
Failed: CWL.TO
12.15% Completed
353 total failures
Failed: CWW.TO
12.18% Completed
354 total failures
Accepted: CWX.TO
12.20% Completed
354 total failures
Failed: CXF.TO
12.23% Completed
355 total failures
Accepted: CXI.TO
12.25% Completed
355 total failures
Failed: CXN.TO
12.28% Completed
356 total failures
Accepted: CXR.TO
12.30% Completed
356 total failures
Failed: CXR.U.TO
12.33% Completed
357 total failures
Failed: CYB.TO
12.36% Completed
358 total failures
Failed: CYBR.TO
12.38% Completed
359 total failures
Failed: CYBR.B.TO
12.41% Completed
360 total failures
Failed: CYH.TO
12.43% Completed
361 total failures
Failed: CZN.TO
12.46% Completed
362 total failures
Faile

Failed: ENB.PF.A.TO
15.96% Completed
470 total failures
Failed: ENB.PF.C.TO
15.99% Completed
471 total failures
Failed: ENB.PF.E.TO
16.01% Completed
472 total failures
Failed: ENB.PF.G.TO
16.04% Completed
473 total failures
Failed: ENB.PF.I.TO
16.07% Completed
474 total failures
Failed: ENB.PF.K.TO
16.09% Completed
475 total failures
Failed: ENB.PF.U.TO
16.12% Completed
476 total failures
Failed: ENB.PF.V.TO
16.14% Completed
477 total failures
Failed: ENB.PR.A.TO
16.17% Completed
478 total failures
Failed: ENB.PR.B.TO
16.19% Completed
479 total failures
Failed: ENB.PR.C.TO
16.22% Completed
480 total failures
Failed: ENB.PR.D.TO
16.24% Completed
481 total failures
Failed: ENB.PR.F.TO
16.27% Completed
482 total failures
Failed: ENB.PR.H.TO
16.30% Completed
483 total failures
Failed: ENB.PR.J.TO
16.32% Completed
484 total failures
Failed: ENB.PR.N.TO
16.35% Completed
485 total failures
Failed: ENB.PR.P.TO
16.37% Completed
486 total failures
Failed: ENB.PR.T.TO
16.40% Completed
487 total f

...

Failed: MCX.V
79.89% Completed
2572 total failures
Failed: MD.V
79.92% Completed
2573 total failures
Failed: MDL.V
79.94% Completed
2574 total failures
Failed: MDV.V
79.97% Completed
2575 total failures
Failed: MDX.V
79.99% Completed
2576 total failures
Failed: MED.V
80.02% Completed
2577 total failures
Failed: MEK.V
80.05% Completed
2578 total failures
Failed: META.V
80.07% Completed
2579 total failures
Failed: MEX.V
80.10% Completed
2580 total failures
Failed: MEZ.H.V
80.12% Completed
2581 total failures
Failed: MFM.V
80.15% Completed
2582 total failures
Failed: MFS.V
80.17% Completed
2583 total failures
Failed: MFX.V
80.20% Completed
2584 total failures
Failed: MGC.H.V
80.23% Completed
2585 total failures
Failed: MGG.V
80.25% Completed
2586 total failures
Failed: MGI.V
80.28% Completed
2587 total failures
Failed: MGM.V
80.30% Completed
2588 total failures
Failed: MGM.WT.V
80.33% Completed
2589 total failures
Failed: MGR.P.V
80.35% Completed
2590 total failures
Failed: MGZ.H.V
80.38%

Failed: NWX.V
83.99% Completed
2725 total failures
Accepted: NXG.V
84.01% Completed
2725 total failures
Failed: NXN.V
84.04% Completed
2726 total failures
Failed: NXO.V
84.06% Completed
2727 total failures
Failed: NXR.UN.V
84.09% Completed
2728 total failures
Failed: NXS.V
84.11% Completed
2729 total failures
Failed: NXT.V
84.14% Completed
2730 total failures
Failed: NZ.V
84.16% Completed
2731 total failures
Failed: NZN.V
84.19% Completed
2732 total failures
Failed: NZP.V
84.22% Completed
2733 total failures
Failed: NZZ.V
84.24% Completed
2734 total failures
Failed: OCC.P.V
84.27% Completed
2735 total failures
Failed: OCN.V
84.29% Completed
2736 total failures
Failed: OCO.V
84.32% Completed
2737 total failures
Failed: OEC.V
84.34% Completed
2738 total failures
Failed: OEE.V
84.37% Completed
2739 total failures
Failed: OG.V
84.39% Completed
2740 total failures
Accepted: OGI.V
84.42% Completed
2740 total failures
Failed: OGI.DB.V
84.45% Completed
2741 total failures
Failed: OGI.WT.A.V
84

Failed: QPT.V
88.05% Completed
2869 total failures
Failed: QRO.V
88.08% Completed
2870 total failures
Accepted: QST.V
88.10% Completed
2870 total failures
Failed: QTA.V
88.13% Completed
2871 total failures
Failed: QUIS.V
88.16% Completed
2872 total failures
Failed: QYOU.V
88.18% Completed
2873 total failures
Failed: RAB.V
88.21% Completed
2874 total failures
Failed: RAK.V
88.23% Completed
2875 total failures
Failed: RAU.H.V
88.26% Completed
2876 total failures
Failed: RB.H.V
88.28% Completed
2877 total failures
Failed: RBE.V
88.31% Completed
2878 total failures
Failed: RBX.V
88.33% Completed
2879 total failures
Failed: RCG.V
88.36% Completed
2880 total failures
Failed: RCK.V
88.39% Completed
2881 total failures
Failed: RCLF.V
88.41% Completed
2882 total failures
Failed: RCT.V
88.44% Completed
2883 total failures
Failed: RDK.H.V
88.46% Completed
2884 total failures
Failed: RDR.P.V
88.49% Completed
2885 total failures
Failed: RDS.V
88.51% Completed
2886 total failures
Failed: RDU.V
88.54

Failed: SMD.V
92.12% Completed
3019 total failures
Failed: SME.V
92.15% Completed
3020 total failures
Failed: SMM.V
92.17% Completed
3021 total failures
Failed: SMN.V
92.20% Completed
3022 total failures
Failed: SMO.V
92.22% Completed
3023 total failures
Failed: SMS.V
92.25% Completed
3024 total failures
Failed: SMY.V
92.27% Completed
3025 total failures
Failed: SN.V
92.30% Completed
3026 total failures
Failed: SNF.V
92.33% Completed
3027 total failures
Failed: SNG.V
92.35% Completed
3028 total failures
Accepted: SNM.V
92.38% Completed
3028 total failures
Failed: SNR.V
92.40% Completed
3029 total failures
Failed: SNS.V
92.43% Completed
3030 total failures
Failed: SNV.V
92.45% Completed
3031 total failures
Failed: SNX.H.V
92.48% Completed
3032 total failures
Failed: SOC.V
92.50% Completed
3033 total failures
Failed: SOCK.V
92.53% Completed
3034 total failures
Failed: SOG.V
92.56% Completed
3035 total failures
Failed: SOI.V
92.58% Completed
3036 total failures
Failed: SOIL.V
92.61% Compl

Failed: UGE.V
96.21% Completed
3170 total failures
Failed: UHO.V
96.24% Completed
3171 total failures
Failed: UI.V
96.27% Completed
3172 total failures
Failed: ULI.V
96.29% Completed
3173 total failures
Failed: UNI.V
96.32% Completed
3174 total failures
Failed: UNO.V
96.34% Completed
3175 total failures
Failed: URC.V
96.37% Completed
3176 total failures
Failed: USS.V
96.39% Completed
3177 total failures
Failed: UTY.V
96.42% Completed
3178 total failures
Failed: UVN.V
96.44% Completed
3179 total failures
Failed: VAN.V
96.47% Completed
3180 total failures
Failed: VANC.V
96.50% Completed
3181 total failures
Failed: VAU.V
96.52% Completed
3182 total failures
Failed: VAX.V
96.55% Completed
3183 total failures
Accepted: VCI.V
96.57% Completed
3183 total failures
Failed: VCOM.V
96.60% Completed
3184 total failures
Failed: VCV.V
96.62% Completed
3185 total failures
Failed: VDO.H.V
96.65% Completed
3186 total failures
Failed: VEIN.V
96.67% Completed
3187 total failures
Failed: VENZ.V
96.70% Com

In [50]:
# earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
# earnings = earnings.drop(earnings.isnull().sum().sort_values().tail(13).index.tolist(), axis = 1)
# earnings.to_csv('full_data.csv')

fs_df = pd.read_csv('full_data.csv', index_col = 0)

fs = fs_df[['Underlying']]
fs['curr_ratio'] = fs_df['totalCurrentAssets']/fs_df['totalCurrentLiabilities']
fs['totaldebtequity'] = fs_df['totalLiab']/fs_df['totalStockholderEquity']
fs['longdebtequity'] = fs_df['longTermDebt']/fs_df['totalStockholderEquity']
fs['shortdebtequity'] = fs_df['shortLongTermDebt']/fs_df['totalStockholderEquity']
fs['daySalesOut'] = (fs_df['netReceivables']/fs_df['totalRevenue'].replace(0,np.nan))*365
fs['intangToBook'] = fs_df['intangibleAssets']/fs_df['totalStockholderEquity']
fs['invToRev'] = fs_df['inventory']/fs_df['totalRevenue'].replace(0,np.nan)
fs['ltdebtToCap'] = fs_df['longTermDebt']/fs_df['investments']
fs['stdebtToCap'] = fs_df['shortLongTermDebt']/fs_df['investments']
fs['ltdebtPercent'] = fs_df['longTermDebt']/fs_df['totalLiab']
fs['stdebtPercent'] = fs_df['shortLongTermDebt']/fs_df['totalLiab']

fs['grossMargin'] = fs_df['grossProfit']/fs_df['totalRevenue'].replace(0,np.nan)
fs['profitMargin'] = fs_df['netIncome']/fs_df['totalRevenue'].replace(0,np.nan)
fs['operatingMargin'] = fs_df['operatingIncome']/fs_df['totalRevenue'].replace(0,np.nan)

fs['operatingCashflow'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalRevenue'].replace(0,np.nan)
fs['assetEfficiency'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalAssets']
fs['currLiabCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalCurrentLiabilities']
fs['ltDebtCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['longTermDebt']
fs['cashGeneratingPower'] = fs_df['totalCashFromOperatingActivities']/(fs_df['totalCashFromOperatingActivities'] + fs_df['totalCashFromFinancingActivities'] + fs_df['totalCashflowsFromInvestingActivities'])

fs_pct = []
for name in fs.Underlying.drop_duplicates().tolist():
    pct = fs[fs['Underlying'] == name].sort_index()
    pct[pct.columns.tolist()[1:]] = pct[pct.columns.tolist()[1:]].pct_change()
    fs_pct.append(pct.tail(len(pct)-1))

fs_pct = pd.concat(fs_pct, axis = 0)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [55]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()

book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

fs_pct.drop_duplicates().to_excel(writer, "finpctdata")
fs.drop_duplicates().to_excel(writer,"findata")
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")
writer.save()

## Checking Key Stats on Single Names
### For each industry filter names that have:
1. positive earnings growth
2. positive profit margin
3. lower than industry average p/b, p/e, ev/ebitda, debt/equity
4. rising margins quarter over quarter
5. decreasing debt/equity quarter over quarter
6. decreasing days of sales outstanding
7. lower than industry average inventory/revenue ratio

In [146]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()
ksavg = pd.read_excel(filename, sheetname = 'keystatsAvg', index_col = 0).drop_duplicates()
finpct = pd.read_excel(filename, sheetname = 'finpctdata', index_col = 0).drop_duplicates().fillna(0)
fin = pd.read_excel(filename, sheetname = 'findata', index_col = 0).drop_duplicates()

# Creating scores of pct changes
tickers = finpct.Underlying.drop_duplicates().tolist()
columns = finpct.columns.tolist()[1:]
values = ['inc','dec','dec','dec','dec','flat','dec','dec','dec',
          'dec','dec','inc','inc','inc','inc','inc','inc','inc',
          'inc']
field_directions = dict(zip(columns,values))
scores = pd.DataFrame(index = tickers, columns = columns)

for name in tickers:
    for field in columns:
        curr_fin = finpct[(finpct['Underlying'] == name)][field]
        if len(curr_fin) > 1:
            if field_directions[field] == 'inc':
                point = len(curr_fin) == sum(curr_fin >= 0)
            elif field_directions[field] == 'dec':
                point = len(curr_fin) == sum(curr_fin <= 0)
            else:
                point = False
        else:
            point = False
            
        scores.loc[name,field] = point*1
        
scores['Score'] = scores.sum(axis = 1)

In [153]:
top_names = scores[scores['Score'] >= 10].sort_values('Score', ascending = False)
ks_col_order = ['currentPrice', 'sector', 'industry',
                '52WeekChange', 'SandP52WeekChange', 'beta', 'bookValue',
                'currentRatio', 'debtToEquity', 'earningsGrowth', 'revenueGrowth', 
                'returnOnAssets', 'returnOnEquity',
                'earningsQuarterlyGrowth', 'heldPercentInsiders', 'heldPercentInstitutions', 
                'sharesPercentSharesOut', 'shortPercentOfFloat', 
                'ebitda', 'ebitdaMargins',
                'enterpriseToEbitda', 'enterpriseToRevenue', 'enterpriseValue',
                'floatShares', 'forwardEps', 'forwardPE', 'freeCashflow',
                'fullTimeEmployees', 'grossMargins', 'grossProfits',
                'netIncomeToCommon', 'operatingCashflow', 'profitMargins','operatingMargins',
                'overallRisk', 'pegRatio', 'priceToBook', 
                'quickRatio', 'recommendationKey',
                'recommendationMean', 
                'revenuePerShare', 'sharesOutstanding',
                'sharesShort', 'sharesShortPriorMonth',
                'shortRatio', 'targetHighPrice',
                'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'totalCash',
                'totalCashPerShare', 'totalDebt', 'totalRevenue', 'trailingEps']

In [154]:
filteredfilename = 'filtered leaps.xlsx'
top_names.to_excel(filteredfilename,'scores')

book = load_workbook(filteredfilename)
writer = pd.ExcelWriter(filteredfilename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

finpct[finpct.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'finpctdata')
fin[fin.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'findata')
ks[ks_col_order][ks.index.isin(top_names.index.tolist())].to_excel(writer,'keystats')
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")

writer.save()



In [173]:
ftnt = yahoo_query('BMO.TO', dt.datetime(2018,1,1))
ftnt.full_info_query()

In [175]:
ftnt.executives

,age,exercisedValue,fiscalYear,name,totalPay,unexercisedValue
title,,,,,,
CEO & Director,45,0,2016,Mr. William Darryl White,3286320,0
Chief Financial Officer,NaN,0,2016,Mr. Thomas E. Flynn,1404960,0
Chief Technology & Operations Officer,NaN,0,2016,Mr. Jean-Michel R. Arès,2538160,0
Group Head of BMO Capital Markets,NaN,0,NaN,Mr. Patrick Cronin,NaN,0
Exec. VP & Chief Information Officer,NaN,0,NaN,Mr. Steve Tennyson,NaN,0


In [176]:
ftnt.purchaseActivity

,buyInfoCount,buyInfoShares,maxAge,netInfoCount,netInfoShares,period,sellInfoCount,totalInsiderShares
BMO.TO,0,0,1,0,0,6m,0,0
